# DRAFY of ETL Process Notebook

# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

# Initial DBCreateion and file check

In [2]:
# from create_tables import *

In [3]:
# cur,conn = create_database()

In [4]:
# conn.close()

# Orig ETL steps

In [5]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
conn.set_session(autocommit=True)
cur = conn.cursor()

In [6]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [7]:
song_files = get_files('./data/song_data/')

In [8]:
song_files[0]

'/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json'

In [9]:
# !head /home/workspace/data/song_data/A/B/B/TRABBVJ128F92F7EAA.json

In [10]:
filepath = song_files[4]

In [11]:
# parse all files to one DataFrame
dfSongs = pd.DataFrame()
for filepath in song_files:
    df = pd.read_json(filepath,lines=True)
    dfSongs = dfSongs.append(df,ignore_index=True)

In [13]:
dfSongs.groupby('artist_name')['artist_id'].count().reset_index()

,artist_name,artist_id
0,40 Grit,1
1,Adam Ant,1
2,Alice Stuart,1
3,Andy Andy,1
4,Backstreet Boys,1
5,Billie Jo Spears,1
6,Bitter End,1
7,Blingtones,1
8,Blue Rodeo,1
9,Bob Azzam,1


In [ ]:
dfSongs[dfSongs.title.str.contains('Dead')]

In [ ]:
dfSongs.columns.tolist()

In [ ]:
# type check
dfSongs.info()

In [ ]:
# dfSongs[[
#     'artist_id',
#     'artist_latitude',
#     'artist_location',
#     'artist_longitude',
#     'artist_name'
# ]].head()#.info()#.drop_duplicates().count()

In [ ]:
dfSongs[
    'artist_id'
].str.len().max()

In [ ]:
dfSongs[
    'artist_name'
].str.len().max()

In [ ]:
def run_sql(sql,cur):
    """
    Simple SQL query execution and Excaption catcher.
    """
    try: 
        cur.execute(sql)
    except psycopg2.Error as e: 
        print("Error: SQL execution")
        print (e)

In [ ]:
from io import StringIO
import csv

def bulk_insert(conn, df, table, use_index=True, index_label='id'):
    """
    Bulk insert of DataFrame to DB using in memory
    buffer and copy_from() to copy it to the table
    Parmas:
    #TODO: to describe
    """
    # save dataframe to an in memory buffer
    buffer = StringIO()
    df.to_csv(buffer, 
              index_label=index_label, 
              header=False,
              index=use_index,
              quoting=csv.QUOTE_NONE,
              sep='|'
             )
    buffer.seek(0)
    
    cursor = conn.cursor()
    try:
        cursor.copy_from(buffer, table, sep="|",null="")
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
#         os.remove(tmp_df)
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("copy_from_stringio() done")
    cursor.close()

In [ ]:
# create artists table
sql = """
CREATE TABLE IF NOT EXISTS artists (
    artist_id varchar(18) PRIMARY KEY,
    artist_name text,
    artist_location text,
    artist_latitude real,
    artist_longitude real
    );
"""

In [ ]:
run_sql(sql,cur)

In [ ]:
sql = """
drop table artists;
"""

In [ ]:
run_sql(sql,cur)

In [ ]:
# dfSongs[[
#     'artist_id',
#     'artist_latitude',
#     'artist_location',
#     'artist_longitude',
#     'artist_name'
# ]]

In [ ]:
# dfSongs

In [ ]:
dfArtists = dfSongs[[
                'artist_id',
                'artist_name',
                'artist_latitude',
                'artist_longitude',
                'artist_location'
            ]].drop_duplicates().copy()

In [ ]:
dfArtists[:3]

In [ ]:
#export to DB
bulk_insert(conn,dfArtists,'artists',use_index=False)

In [ ]:
# for idx,row in dfSongs[[
#     'artist_id',
#     'artist_name',
#     'artist_latitude',
#     'artist_longitude',
#     'artist_location'
# ]].drop_duplicates().iterrows():
#     sql = f"""
#     INSERT INTO artists (artist_id, artist_name, artist_latitude, artist_longitude, artist_location) \
#                  VALUES (
#                  '{row['artist_id']}',
#                  '{row['artist_name']}',
#                  {row['artist_latitude']},
#                  {row['artist_longitude']},
#                  '{row['artist_location']}'
#                  )
#     """
# #     print(sql)
#     run_sql(sql,cur)
# #     break
# #.to_sql('artists',conn,if_exists='append',index=False)

In [ ]:
# songs

In [ ]:
dfSongs[[
    'song_id',
    'title',
    'duration',
    'num_songs', #?? what for
    'year' 
]].describe()#.drop_duplicates().count()

In [ ]:
dfSongs.head()

In [ ]:
# any duplicate chack
# dfSongsArtistsGrouping = dfSongs.groupby('artist_id')['num_songs'].count()
dfSongsArtistsGrouping = dfSongs.groupby('artist_id')['num_songs'].sum()
dfSongsArtistsGrouping[dfSongsArtistsGrouping>1]

In [ ]:
# checking datatypes before table creation
dfSongs[[
    'song_id',
    'title',
    'artist_id',
    'year',
    'duration'
    ]].info()

In [ ]:
# what is max len for varchar?
dfSongs[
    'song_id'
].str.len().max()

In [ ]:
# create sons table
sql = """
CREATE TABLE IF NOT EXISTS songs (
    song_id varchar(18) PRIMARY KEY,
    title text,
    artist_id varchar(18),
    year int,
    duration real
    );
"""

In [ ]:
run_sql(sql,cur)

In [ ]:
sql = """
drop table songs;
"""

In [ ]:
run_sql(sql,cur)

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [ ]:
dfSongs.columns

In [ ]:
dfSongs[[
    'song_id',
    'title',
    'artist_id',
    'year',
    'duration'
    ]].info()

In [ ]:
song_data = dfSongs[[
    'song_id',
    'title',
    'artist_id',
    'year',
    'duration'
    ]].copy()

In [ ]:
song_data = song_data.values[0].tolist()

In [ ]:
song_data

In [ ]:
# print("INSERT INTO songs (song_id, title, artist_id, year, duration) \
# VALUES (%s, %s, %s, %d, %f)" % tuple(song_data.values[0].tolist()))

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [ ]:
song_table_insert

In [ ]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [ ]:
artist_data = dfSongs[[
    'artist_id',
    'artist_name',
    'artist_location',
    'artist_latitude',
    'artist_longitude'
    ]].values[0].tolist()
artist_data

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [ ]:
print("INSERT INTO artists (artist_id, artist_name, artist_location, artist_latitude, artist_longitude) \
                    VALUES (%s, %s, %s, %s, %s)" % tuple(artist_data))

In [ ]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [15]:
log_files = get_files('./data/log_data/')

In [16]:
filepath = log_files[0]

In [17]:
dfLogData = pd.DataFrame()
for filepath in log_files:
    df = pd.read_json(filepath,lines=True)
    dfLogData = dfLogData.append(df,ignore_index=True)

In [20]:
dfLogData[dfLogData['artist'].isin(['Tesla'])]

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId


In [15]:
dfLogData.groupby('artist')['song'].count().reset_index()

,artist,song
0,!!!,1
1,'N Sync/Phil Collins,1
2,+ / - {Plus/Minus},2
3,+44,1
4,1 Mile North,1
5,10 Years,1
6,10_000 Maniacs,1
7,10cc,3
8,12 Stones,3
9,1200 Mics,1


In [ ]:
dfLogData.head()

In [ ]:
df = pd.read_json(filepath,lines=True)
df.head()

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
df.page.unique()

In [ ]:
df.ts = pd.to_datetime(df.ts,unit='ms')

In [ ]:
df[df.page == 'NextSong'] #.columns.tolist()

In [ ]:
df = df[df.page == 'NextSong']
df.head()

In [ ]:
t = pd.to_datetime(df.ts,unit='ms')
t.head()

In [ ]:
# t.dt.time
# t.dt.hour
# t.dt.day
# t.dt.weekofyear
# t.dt.month
# t.dt.year
# t.dt.weekday
# t.values.astype('int') #???

In [ ]:
# t.astype('int')
# (t.astype('int64')/1e6).astype('int')

In [ ]:
time_data = (
#     t.astype(pd.Timestamp),
    (t.astype('int64')/1e6).astype('int'),
    t.dt.hour,
    t.dt.day,
    t.dt.weekofyear,
    t.dt.month,
    t.dt.year,
    t.dt.weekday
    )
column_labels = [
    'timestamp',
    'hour',
    'day',
    'weekofyear',
    'month',
    'year',
    'weekday'
]

In [ ]:
# time_data[0].tolist()

In [ ]:
# time_df = pd.DataFrame(dict(zip(column_labels,[x.tolist() for x in time_data])))

In [ ]:
# pd.DataFrame(time_data)

In [ ]:
time_df = pd.DataFrame(dict(zip(column_labels,[x.tolist() for x in time_data])))
time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
time_df.info()

In [ ]:
# Crating SQL for create table

In [ ]:
# create artists table
sql = """
CREATE TABLE IF NOT EXISTS time (
    timestamp TIMESTAMP,
    hour smallint,
    day smallint,
    weekofyear smallint,
    month smallint,
    year smallint,
    weekday smallint
    );
"""

In [ ]:
run_sql(sql,cur)

In [ ]:
sql = """
drop table time;
"""

In [ ]:
run_sql(sql,cur)

In [ ]:
time_data = time_df.values[0].tolist()

In [ ]:
print("INSERT INTO time (timestamp, hour, day, weekofyear, month, year, weekday) \
                    VALUES (%s, %s, %s, %s, %s, %s, %s)" % tuple(time_data))

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
dfLogData.columns.tolist()

In [ ]:
user_df = dfLogData[[
    'userId',
    'firstName',
    'lastName',
    'gender',
    'level'
]].copy()

In [ ]:
user_df

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
user_df.columns.tolist()

In [ ]:
user_df.info()

In [ ]:
user_df.userId.unique()

In [ ]:
user_df.gender.unique()

In [ ]:
user_df.level.unique()

In [ ]:
# create artists table
sql = """
CREATE TABLE IF NOT EXISTS users (
    userId smallint PRIMARY KEY,
    firstName text,
    lastName text,
    gender varchar(1),
    level varchar(4) 
    );
"""

In [ ]:
run_sql(sql,cur)

In [ ]:
user_data = user_df.values[0].tolist()

In [ ]:
print("INSERT INTO users (userId, firstName, lastName, gender, level) \
                    VALUES (%s, %s, %s, %s, %s)" % tuple(user_data))

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
log_files = get_files('./data/log_data/')

In [ ]:
# filepath = log_files[0]

In [ ]:
dfLogData = pd.DataFrame()
for filepath in log_files:
    df = pd.read_json(filepath,lines=True)
    dfLogData = dfLogData.append(df,ignore_index=True)
df = dfLogData.copy()

In [ ]:
df.info()

In [ ]:
# unification of timestamp
df.ts = pd.to_datetime(df.ts,unit='ms')

In [ ]:
df.userId = df.userId.replace('',-1)

In [ ]:
# df.ts

In [ ]:
# analysis of data and selection to log 

In [ ]:
df[[
#     'artist',
    'userId',
    'auth',
#     'firstName',
#     'gender',
    'itemInSession',
#     'lastName',
#     'length',
    'level',
    'location',
    'method',
    'page',
    'registration',
    'sessionId',
#     'song',
    'status',
    'ts',
    'userAgent',    
]].info()#head(3)

In [ ]:
# df.level.unique()#str.len().max()
# df.sessionId

In [ ]:
# df[~df.registration.isna()].registration.astype('int64')
# df.registration = df.registration.fillna(-1)
# df.registration = df.registration.astype('int64')
# df[df.registration != df.ts][['registration','ts']]
# registration and timestamp different so let's log

In [ ]:
# print("select song_id, songs.artist_id from \
#         songs join artists on songs.artist_id = artists.artist_id \
#         where songs.title = '%s' and artists.artist_name = '%s' and songs.duration = %s;\
#       " % ("I Didn't Mean To","Casual",218.932))

In [ ]:
# print("select song_id, songs.artist_id from \
#         songs join artists on songs.artist_id = artists.artist_id \
#         where artists.artist_name = '%s' and songs.duration = %s;\
#       " % ("Casual",218.932))

In [ ]:
# song_select = ("""
# select song_id, songs.artist_id
# from songs join artists on songs.artist_id = artists.artist_id
# where songs.title = %s
#       and artists.artist_name = %s
#       and round(songs.duration::numeric,0) = round(%s::numeric,0);
# """)

In [ ]:
# song_select

In [ ]:
cur.execute(song_select, ("I Didn't Mean To","Casual",218.93179))
results = cur.fetchone()
print(results)

In [ ]:
# so proposal for table collumns
# sql = """
# CREATE TABLE IF NOT EXISTS songplays (
#     timestamp TIMESTAMP, 
#     userId smallint,
#     level varchar(4), 
#     song_id varchar(18),
#     artist_id varchar(18),
#     session_id smallint,
#     location text,
#     userAgent text 
# """

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (
        row.ts, 
        row.userId,
        row.level, 
        songid,
        artistid,
        row.sessionId,
        row.location,
        row.userAgent
    )
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.